In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly.express as px
from openai import OpenAI
from google.colab import userdata
from dataclasses import dataclass, field

In [ ]:
api_key = userdata.get('openai_api_key')
openai = OpenAI(api_key=api_key)
model = 'gpt-4o-2024-05-13'

In [30]:
@dataclass
class BossAgent:

    start_word: str = 'こんにちは'
    model = model
    openai = openai
    messages: list = field(default_factory=list)

    @property
    def system_call(self):
        system_call = '''
        あなたは優秀なデータ分析課の上司です。解答を急がず、
        部下の分析をより広い視野から発送できる、データからよい意思決定を導ける、課題を深堀できる、
        データからエビデンスを導ける、指示やアドバイスを出してください。
        '''.strip()
        return system_call

    def chat(self, prompt: str):
        if self.messages == []:
            sys = self.system_call + f'最初の指示: {self.start_word}'
            self.messages.append({'role': 'system', 'content': sys})
        self.messages.append({'role': 'user', 'content': prompt})
        try:
            res = openai.chat.completions.create(
                model=self.model,
                messages=self.messages,
            )
            self.messages.append({'role': 'assistant', 'content': res.choices[0].message.content})
            return res.choices[0].message.content
        except Exception as e:
            print(e)
            return None


@dataclass
class DataAnalyst:

    model = model
    openai = openai
    messages: list= field(default_factory=list)

    @property
    def system_call(self):
        system_call = """
        あなたは優秀なデータ分析官です。結論を急がず、自分の考えと
        上司の指示を基に、より課題を深く検討し、
        データ分析を行ってください。

        """
        return system_call


    def analysis(self, prompt: str):
        if self.messages == []:
            self.messages.append({'role': 'system', 'content': self.system_call})
        self.messages.append({'role': 'user', 'content': prompt})
        chat = openai.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        self.messages.append({'role': 'assistant', 'content': chat.choices[0].message.content})
        return chat.choices[0].message.content

In [31]:
df = pd.read_csv('/content/drive/MyDrive/work/pycon_kyushu/data/model-eval-price.csv')
start_word = f''' 次のデータから、どのモデルが、プロダクトテストに使うのに割安で良いかを分析してください。
    データ: {df}
'''
boss = BossAgent(start_word)
data_analyst = DataAnalyst()


In [32]:
res1 = data_analyst.analysis(start_word)

In [33]:
print(res1)

プロダクトテストに使うのに最もコストパフォーマンスが良いモデルを選ぶためには、パフォーマンス（MMLU、MATH）と価格のバランスを考慮する必要があります。以下の手順で分析を進めます。

### ステップ1: データの確認
データ内の各モデルの情報を確認します。

### ステップ2: 標準化のための指標設定
価格が異なるため、特定の指標を使って各モデルの価格対性能比を比較します。

例えば、各モデルのMMLUとMATHのパフォーマンススコアの合計を元に割安度を算出します。

### パフォーマンス/価格比の計算
それぞれのモデルについて、性能の合計（MMLU + MATH）を価格で割ることで、性能/価格比を求めます。

```plaintext
性能の合計/価格 = (MMLU + MATH) / Price
```

### ステップ3: 比率を計算して比較
比率を計算し、それに基づいて各モデルを比較します。

以下は、計算結果です:

```python
import pandas as pd

# データフレームの作成
data = {
    'model': ['GPT-3.5', 'GPT-4', 'GPT-4 Turbo', 'GPT-4o',
              'Claude 3-Haiku', 'Claude 3-Opus', 'Gemini-1.5-PRO', 'Gemini-1.5-FLASH'],
    'MMLU(5shot)': [70, 86.4, 86.5, 88.7, 75.2, 86.8, 81.9, 78.9],
    'MATH': [34.10, 52.90, 72.60, 76.60, 38.90, 60.10, 58.50, 54.90],
    'Price': [1.5, 60, 30, 15, 1.25, 75, 10.5, 0.53]
}

df = pd.DataFrame(data)

# 性能合計と性能/価格比の計算
df['Performance'] = df['MMLU(5shot)'] + df['MATH']
df['Performance/Price'] = df['Performance'] / df['Price']

# Print the results
print(df

In [34]:
res2 = boss.chat(res1)
print(res2)

とてもよいアプローチです。データの確認、パフォーマンス/価格比の計算、そして比較をステップごとに行ったことにより、価格対性能比をしっかりと評価できました。では、更に深掘りして、他の要素も考慮してみましょう。

### ステップ 5: 時系列データの考慮
リリース日を考慮すると、新しいモデルは古いものよりも性能が高い傾向があります。また、新しいモデルは今後のサポートやアップデートが期待できるため、リリース日も一つの重要な要素と考えましょう。

```python
# リリース日データを日付型に変換
df['release date'] = pd.to_datetime(df['release date'])

# リリース日からの経過時間（現在の日付を基準に）を計算
current_date = pd.to_datetime('2023/12/31')
df['days_since_release'] = (current_date - df['release date']).dt.days

# 結果を表示
print(df[['model', 'Performance', 'Price', 'Performance/Price', 'days_since_release']])
```

### 追加分析 - コストパフォーマンス指標の加重平均
パフォーマンス/価格比に具体的なウェイト（例：パフォーマンス優先度や価格優先度）を設定してより柔軟な指標を計算します。

```python
# パフォーマンス（MMLUとMATHの加重合計）の重みを設定（例：MMLUをより重要視する）
w_mmlu = 0.7
w_math = 0.3
df['Weighted_Performance'] = df['MMLU(5shot)'] * w_mmlu + df['MATH'] * w_math

# 性能/価格比の計算
df['Weighted_Performance/Price'] = df['Weighted_Performance'] / df['Price']

# 結果を表示
print(df[['model', 'Weighted_Performance', 'Price', 'Weighted_Performance/Price', 'days_since_re

In [36]:
res3 = data_analyst.analysis(res2)
print(res3)

ご指摘の通り、リリース日やその他の要因を考慮に入れることで、より詳細な分析を行うことができます。以下に追加分析を行いました。

### ステップ 5: 時系列データの考慮
リリース日を基に、新しいモデルがどれくらい新しいかを計算します。

```python
import pandas as pd

# データフレームの作成
data = {
    'model': ['GPT-3.5', 'GPT-4', 'GPT-4 Turbo', 'GPT-4o',
              'Claude 3-Haiku', 'Claude 3-Opus', 'Gemini-1.5-PRO', 'Gemini-1.5-FLASH'],
    'MMLU(5shot)': [70, 86.4, 86.5, 88.7, 75.2, 86.8, 81.9, 78.9],
    'MATH': [34.10, 52.90, 72.60, 76.60, 38.90, 60.10, 58.50, 54.90],
    'Price': [1.5, 60, 30, 15, 1.25, 75, 10.5, 0.53],
    'release_date': ['2022/12', '2023/3/14', '2023/12/1', '2024/5/13', '2024/3/4', '2024/3/4', '2024/2/15', '2024/5/14']
}

df = pd.DataFrame(data)

# リリース日データを日付型に変換
df['release_date'] = pd.to_datetime(df['release_date'])

# リリース日からの経過時間（現在の日付を基準に）を計算
current_date = pd.to_datetime('2023/12/31')
df['days_since_release'] = (current_date - df['release_date']).dt.days

# Performanceの計算
df['Performance'] = df['MMLU(5shot)'] + df['MATH']

# Performance/Priceの計算
df['Performance/Price'] = df

In [37]:
df = pd.read_csv('/content/drive/MyDrive/work/pycon_kyushu/data/model-eval-price.csv')
start_word = f''' 次のデータから、どのモデルが、データ分析機能を持つアプリケーションのプロダクトテストに使うのに割安で良いかを分析してください。
    データ: {df}
'''
boss = BossAgent(start_word)
data_analyst = DataAnalyst()


In [38]:
res1 = data_analyst.analysis(start_word)
print(res1)

データを検討する際に考慮すべき要素として、性能（MMLUとMATHのスコア）、価格、およびリリース日があります。それぞれのモデルを比較するために、上記の要素を総合的に評価します。

まず、データ全体を確認して、性能やコストパフォーマンスの観点から有利と思われるモデルを洗い出します。

#### 1. モデル別の性能比較:
- **MMLU**: GPT-4o > Claude 3-Opus > GPT-4 Turbo ≒ GPT-4 > Claude 3-Haiku > Gemini-1.5-PRO > Gemini-1.5-FLASH > GPT-3.5
- **MATH**: GPT-4o > GPT-4 Turbo > Claude 3-Opus > GPT-4 > Gemini-1.5-PRO > Gemini-1.5-FLASH > Claude 3-Haiku > GPT-3.5

#### 2. コストパフォーマンスを考えた評価
各モデルについて、性能と価格のバランスを指数化して評価します。まずは、各モデルの代表的な性能スコア（仮にMMLUとMATHの平均値を採用）を価格で割って、コストパフォーマンスの指標を算出します。

- **GPT-3.5**: (70% + 34.10%) / 2 =  52.05 / 1.50 ≈ 34.70
- **GPT-4**: (86.40% + 52.90%) / 2 = 69.65 / 60.00 ≈ 1.16
- **GPT-4 Turbo**: (86.50% + 72.60%) / 2 = 79.55 / 30.00 ≈ 2.65
- **GPT-4o**: (88.70% + 76.60%) / 2 = 82.65 / 15.00 ≈ 5.51
- **Claude 3-Haiku**: (75.20% + 38.90%) / 2 = 57.05 / 1.25 ≈ 45.64
- **Claude 3-Opus**: (86.80% + 60.10%) / 2 = 73.45 / 75.00 ≈ 0.98
- **Gemini-1.5-PRO**: (81.90% + 58.50%) / 2 = 70.20 / 10.50 ≈ 6.69
- **Gemini-1.5-FLASH**: (78.90% + 

In [39]:
res2 = boss.chat(res1)
print(res2)

あなたが提示した分析は、性能とコストパフォーマンスのバランスを重視し、モデルを評価するための有効な一手法です。ただし、以下の点をさらに詳細に検討することで、より深い洞察が得られる可能性があります。

1. **性能の重み付け**:
   各アプリケーションにおいて、MMLUとMATHのどちらのスコアがより重要かに応じて、性能評価の重みを変えることが有効です。例えば、数値計算性能が重要なアプリケーションなら、MATHのスコアにより重きを置くべきです。

2. **価格のスケーリング**:
   モデルの導入にかかるコストは、使用頻度や利用規模によって大きく異なります。価格を評価する際に、実際の使用ケース（例えば、月間のAPIコール数）に基づいて検討することが重要です。

3. **リリース日の考慮**:
   最新のモデルは通常改善された機能を提供する可能性がありますが、リリース直後はバグやサポートの問題が発生するリスクも考慮すべきです。この点も場合によっては重要です。

### 追加分析:
以下は、上述のポイントに基づく追加分析の例です：

#### 1. 重み付けを考慮した総合評価：
MMLUとMATHの性能を等しく重視する場合には現行の方法で問題ありませんが、特定の要件に応じた重み付けを適用します（例: MATH 70%、MMLU 30%）。
```python
models = [
    {"name": "GPT-3.5", "MMLU": 70, "MATH": 34.10, "price": 1.50},
    {"name": "GPT-4", "MMLU": 86.40, "MATH": 52.90, "price": 60.00},
    {"name": "GPT-4 Turbo", "MMLU": 86.50, "MATH": 72.60, "price": 30.00},
    {"name": "GPT-4o", "MMLU": 88.70, "MATH": 76.60, "price": 15.00},
    {"name": "Claude 3-Haiku", "MMLU": 75.20, "MATH": 38.90, "price": 1.25},
    {"name": "Claude 3-Opus", "MMLU": 

In [40]:
res3 = data_analyst.analysis(res2)
print(res3)

あなたの指摘に基づいて、追加分析および詳細な考察を進めます。特に、MMLUとMATHの性能に対して重み付けを行い、実際の運用コスト評価を行い、新しいモデルのリリース日に関する安定性やサポートのリスクについても考慮します。

### 1. 重み付けを考慮した総合評価
たとえば、数値計算性能（MATH）が70%、総合的な知識性能（MMLU）が30%の比率で重要と仮定します。その計算を以下に示します。

```python
models = [
    {"name": "GPT-3.5", "MMLU": 70, "MATH": 34.10, "price": 1.50},
    {"name": "GPT-4", "MMLU": 86.40, "MATH": 52.90, "price": 60.00},
    {"name": "GPT-4 Turbo", "MMLU": 86.50, "MATH": 72.60, "price": 30.00},
    {"name": "GPT-4o", "MMLU": 88.70, "MATH": 76.60, "price": 15.00},
    {"name": "Claude 3-Haiku", "MMLU": 75.20, "MATH": 38.90, "price": 1.25},
    {"name": "Claude 3-Opus", "MMLU": 86.80, "MATH": 60.10, "price": 75.00},
    {"name": "Gemini-1.5-PRO", "MMLU": 81.90, "MATH": 58.50, "price": 10.50},
    {"name": "Gemini-1.5-FLASH", "MMLU": 78.90, "MATH": 54.90, "price": 0.53},
]

weighted_scores = []
for model in models:
    weighted_score = (((model["MMLU"] * 0.3) + (model["MATH"] * 0.7)) / model["price"])
    weighted_scores.append((model["name"], weighted_s